In [7]:
!pip install rasterio
!pip install tensorflow
!pip install pyrsgis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import os 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import rasterio as rio
from pyrsgis import raster
from pyrsgis.convert import changeDimension, array_to_table
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [9]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [71]:
imagens_bq = [];
imagens_rioVerde = [];
data_dir = "/content/gdrive/MyDrive/dados_redes_neurais_imagens/classes"

for image in os.listdir(os.path.join(data_dir, 'imgs_RioVerde_NN')):
        image_path = os.path.join(data_dir, 'imgs_RioVerde_NN', image)
        img = rio.open(image_path).read()
        imagens_rioVerde.append((img, 0));
        #show(img)

for image in os.listdir(os.path.join(data_dir, 'imgs_BQ_NN')):
        image_path = os.path.join(data_dir, 'imgs_BQ_NN', image)
        img = rio.open(image_path).read()
        imagens_bq.append((img, 1));
        #show(img)

In [72]:
imagens_bq = [np.asarray(img) for img in imagens_bq]
imagens_rioVerde = [np.asarray(img) for img in imagens_rioVerde]

#ESSA FUNÇÃO TRASNFORMA O SHAPE DE (3, 217, 317) PARA (68789, 3)
#imagens_bq = [array_to_table(img) for img in imagens_bq]
#imagens_rioVerde = [array_to_table(img) for img in imagens_rioVerde]
imagens_bq[0].shape

(2,)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(imagens_bq, imagens_rioVerde[0:21], test_size = 0.4)

In [75]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [76]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [77]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 30, 30, 16)      

In [78]:
logdir='logs'

In [79]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [80]:
hist = model.fit(X_train, epochs=20, validation_data=y_train)

ValueError: ignored